In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!mkdir -p output/yellow
!python -V

Python 3.11.9


In [3]:
import pickle
import pandas as pd

year = 2023
month = 3
output_file = f'output/yellow/{year:04d}-{month:02d}.parquet'

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/nix/store/29vig1xp5a0mamssg4sj5xylprwj3j1x-python3.11-scikit-learn-1.4.2/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
print('Standard deviation of the predicted duration: ', round(y_pred.std(), 2))

Standard deviation of the predicted duration:  6.25


In [9]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [10]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred

In [11]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
import os

file_size_mb = (os.path.getsize(output_file)) / 1024 / 1024
print('File size MB: ', round(file_size_mb, 2))

File size MB:  65.46


In [13]:
# Convert this notebook to the python script
#!jupyter nbconvert --to script deployment/homework/starter.ipynb